## Instruccions

Edita, elabora i executa els comandaments i respon a les preguntes en aquest mateix document. Però **atenció!** l'has de descargar (en format `.ipynb`) per poder lliurar-lo a través de l'aula virtual.

## Problema 1
Un centre de seqüenciació t'envia alguns fitxers de seqüencies curtes, que corresponen a nou mostres, de tres espècies diferents, identificades amb els codis *sot*, *cav* i *rei*. Els arxius *fastq* estan disponibles en la carpeta `data`, junt amb altres fitxers en formats alternatius. Primer, hauràs d'identificar els fitxers que contenen les seqüències curtes en format *fastq*. A continuació, tria entre els tres blocs de codi següents quin necessites adaptar per analitzar la qualitat de les seqüències. Per últim, corregeix-lo si conté errors i adapta'l per produir l'anàlisi de qualitat de les seqüències curtes d'aquells fitxers.

In [ ]:
## Opción 1

suppressMessages(library('ShortRead'))
ArchivosFastq <- dir('data', pattern = '*.at')
Calidades <- lapply(ArchivosFastq, function(x) {
    lecturas <- readFastq('data', pattern = x)
    return(width(lecturas))
})
plot(c(0,140), c(0, 0.04), type='n', xlab='Quality?', ylab='Frequency')
x <- 1
for (i in Calidades) {
    lines(density(i), col=x)
    x <- x + 1
}

In [ ]:
## Opción 2

suppressMessages(library('ShortRead'))
ArrosAlForn <- qa('data', type = 'fastq', pattern = '*.fastq',
                  BPPARAM = BiocParallel::registered()[3])
report(QualityReport, dest = 'InformeQualitat.html')

In [ ]:
## Opción 3

suppressMessages(library('ShortRead'))
archivos <- paste0(rep(c('cav','sot','rei'), each = 3), rep(1:3, 3), '.fastq')
archivos
calidadMedia <- numeric(length = 9)
longitudMedia <- numeric(length = 9)
for (i in 1:9) {
    datos <- readFastq('data', pattern = archivos[i])
    calidadMedia[i] <- mean(as(quality(datos), 'numeric'))
    longitudMedia[i] <- mean(width(datos))
}
data.frame(archivos, calidadMedia, longitudMedia)

Contesta les preguntes següents:

1. Has esbrinat quina extensió correspon al format *fastq* en la carpeta `data`?
2. Per a què serveix la funció `width()`?
2. Fes un breu resum de les conclusions més importants de l'anàlisi de qualitat, indicant si creus que la qualitat és suficient i si la qualitat és homogènia entre les mostres.
3. Entre les nou mostres, en trobes alguna que destaque en algun aspecte de la resta?
4. Pots indicar quina és la longitud mitjana de les lectures en cada arxiu?
5. Quina seria la probabilitat de què una "C" en certa posició d'una lectura curta en realitat no fóra una citosina en la molècula de la qual procedeix eixa lectura, si la seua qualitat és "J", és a dir 41?

## Problema 2

A continuació pots executar un exemple de consulta de la base de dades *European Nucleotide Archive*. Aquest exemple, a més de descarregar els resultats de la consulta en un arxiu de text, els llig en R i obté alguns estadístics. Comprova que funciona i familiaritzat amb el codi abans de respondre a les preguntes següents i resoldre el problema plantejat. En la pàgina [taxonomía del NCBI](https://www.ncbi.nlm.nih.gov/taxonomy) trobaràs els codis **taxid** de qualsevol espècie o grup taxonòmic. Per exemple:

 |Espècie        |    taxid|
 |-------------- | --------|
 |*Homo sapiens* |     9606|
 |*Mus musculus* |    10090|
 |Virus          |    10239|
 |Bacteris       |        2|

In [ ]:
library(curl)
portal   <- 'https://www.ebi.ac.uk/ena/portal/api/'
endpoint <- 'search?'
result   <- 'result=read_run'
query    <- paste0('&query=',
                   URLencode('tax_tree(2) AND first_public>=2021-05-01',
                             reserved=TRUE))
fields   <- '&fields=scientific_name,read_count,base_count,library_source,center_name,country'
limit    <- '&limit=0'

URL <- paste0(portal, endpoint, result, query, fields, limit, sep='', collapse='')
curl_download(URL, destfile = 'descarga1.tsv')
resultados <- read.table('descarga1.tsv', header = TRUE, quote = "\"",
                         sep = '\t', na.string = '', stringsAsFactors = FALSE)
head(resultados)
dim(resultados)
sum(resultados$read_count, na.rm = TRUE)
sum(resultados$base_count, na.rm = TRUE)
resultados$genero <- sapply(resultados$scientific_name, function(x) strsplit(x, ' ')[[1]][1])
head(resultados)
sort(table(resultados$genero), decreasing = TRUE)
resultados$country <- sapply(resultados$country, function(x) strsplit(x, ':')[[1]][1])
sort(table(resultados$country), decreasing = TRUE)
sort(table(resultados[resultados$country == 'Russia', 'scientific_name']), decreasing = TRUE)
sort(table(resultados[resultados$country == 'Ireland', 'center_name']), decreasing = TRUE)

#### Preguntes sobre l'anàlisi d'exemple anterior

1. Quin és el nombre de registres d'experiments de seqüenciació amb lectures curtes publicats en ENA durant aquest mes de maig?

2. Quin és el gènere de bacteris més seqüenciat mitjançant lectures curtes en el mes de maig del present any?

3. Quines espècies de bacteris s'han dedicat a seqüenciar en Rússia durant el mes de maig? I en Espanya?

4. Quants centres d'investigació espanyols diferents han enviat seqüències curtes de bacteris a la base de dades ENA durant este mes? I irlandesos? 


####  Problema
Elabora ara la teua pròpia consulta, amb un bloc de codi nou, per investigar els estudis de seqüenciació amb lectures curtes en el ratolí (*Mus musculus*, taxid 10090) durant el mes de maig del present any. Redacta un xicotet resum amb les teues conclusions. Hauries d'incloure informació sobre: els tipus de teixits o línies cel·lulars més seqüenciats, els tipus d'estudi (*library_source*), el balanç entre ratolins mascle i femella seqüenciats, així com qualsevol altre aspecte que et cride l'atenció. Si prefereixes realitzar l'anàlisi en [la web de ENA](https://www.ebi.ac.uk/ena/browser/home), o bé en un full de càlcul després de descarregar-ne els resultats, també podràs obtenir la nota màxima d'aquest apartat. Però procura no demorar-te més de 20 minuts ací.


## Problema 3

Per poder realitzar aquest problema, necessites haver instal·lat els programes blast del NCBI. Per fer-ho, només cal executar el codi següent. Pot tardar uns segons.

In [ ]:
system2(command = './preparar_ambiente.sh', wait = TRUE)

Dins la carpeta `data` tens un arxiu fasta anomenat `a.fa`. L'objectiu és realitzar algunes cerques de blast amb eixa *query*. Pots basar-te en els comandaments del bloc següent, sempre que pugues corregir-ne els errors.

1. Busca la proteïna **a** en la base de dades *swissprot* (instal·lada en la carpeta actual) mitjançant **blastp**, utilitzant un llindar de valor E de 0.001. Pren nota del nombre de resultats obtinguts.
3. Coneixent la definició del "valor E" i a la vista dels resultats de la cerca, sabries dir quants dels resultats obtinguts esperes que siguen casuals (és a dir, que igualment podrien haver-se produït a l'atzar)?
4. Realitza ara una cerca amb **psiblast**, en la mateixa base de dades i amb la mateixa proteïna de partida. Quantes iteracions són necessàries per abastar la convergència si fas servir un llindar d'inclusió (`-inclusion_ethresh`) de 0.001?
5. Compara el nombre de resultats obtinguts amb **psiblast** amb el nombre de resultats obtinguts abans amb **blastp** i explica a què és deguda la diferència.

In [ ]:
system2(command = 'blastn',
        args = c('-db', 'swissprot',
                 '-query', 'data/b.fa',
                 '-out', 'psi1.txt',
                 '-outfmt', '"7 qaccver saccver pident length qstart qend sstart send
   evalue ssciname sblastname"',
                 '-evalue', '0.005',
                 '-inclusion_ethresh', '0.002',
                 '-num_iterations', '0'),
       stdout=TRUE)